In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255

y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

In [4]:
x_letter = train['letter'].values
x_letter = x_letter[:, np.newaxis]
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()
x_letter.shape

(2048, 26)

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

x_letter_train = x_letter[:x_train.shape[0],:]
x_letter_val = x_letter[x_train.shape[0]:,:]
print(x_letter_train.shape)
print(x_letter_val.shape)

(1638, 28, 28, 1)
(410, 28, 28, 1)
(1638, 10)
(410, 10)
(1638, 26)
(410, 26)


In [8]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
______________________________________________________________________________________________

In [10]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
history = model.fit([x_train, x_letter_train], y_train, validation_data=([x_val, x_letter_val], y_val), 
                    batch_size=64, epochs=50, verbose=1)

Train on 1638 samples, validate on 410 samples
Epoch 1/50
1638/1638 [==============================] - 4s 2ms/sample - loss: 2.2989 - accuracy: 0.1197 - val_loss: 2.2906 - val_accuracy: 0.1220
Epoch 2/50
1638/1638 [==============================] - 3s 2ms/sample - loss: 2.2466 - accuracy: 0.1600 - val_loss: 2.2216 - val_accuracy: 0.2707
Epoch 3/50
1638/1638 [==============================] - 3s 2ms/sample - loss: 2.1223 - accuracy: 0.2161 - val_loss: 2.1404 - val_accuracy: 0.3341
Epoch 4/50
1638/1638 [==============================] - 3s 2ms/sample - loss: 1.8939 - accuracy: 0.3266 - val_loss: 1.9277 - val_accuracy: 0.4293
Epoch 5/50
1638/1638 [==============================] - 3s 2ms/sample - loss: 1.6662 - accuracy: 0.4103 - val_loss: 1.7010 - val_accuracy: 0.5171
Epoch 6/50
1638/1638 [==============================] - 3s 2ms/sample - loss: 1.4794 - accuracy: 0.4945 - val_loss: 1.5329 - val_accuracy: 0.5854
Epoch 7/50
1638/1638 [==============================] - 3s 2ms/sample - loss:

In [13]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255

x_letter_test = test['letter'].values
x_letter_test = x_letter_test[:, np.newaxis]
en = OneHotEncoder()
x_letter_test = en.fit_transform(x_letter_test).toarray()
x_letter_test.shape

(20480, 26)

In [14]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(model.predict([x_test, x_letter_test]), axis=1)

In [15]:
submission.to_csv('data/submission1.csv', index=False)